In [1]:
import datetime
import os
from mdagent import MDAgent

In [2]:
prompt14 = "Simulate oxygenated hemoglobin (1A3N) and deoxygenated hemoglobin (6BB5)."
llm_var = "accounts/fireworks/models/llama-v3p1-405b-instruct"
tools = "all"
agent = MDAgent(agent_type="Structured", model=llm_var, top_k_tools=tools)
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date and time:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)
print("LLM: ",agent.llm.model_name,"\nTemperature: ",agent.llm.temperature)

date and time: 2024-09-07
time: 18:54:50
LLM:  accounts/fireworks/models/llama-v3p1-405b-instruct 
Temperature:  0.1


In [3]:
agent.run(prompt14)

Thought: To simulate oxygenated hemoglobin (1A3N) and deoxygenated hemoglobin (6BB5), I need to set up and run a molecular dynamics simulation for each protein. 

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "1A3N",
    "forcefield_files": ["amber99sbildn.xml", "tip3p.xml"],
    "save": true,
    "system_params": {
      "nonbondedMethod": "NoCutoff",
      "constraints": "None",
      "rigidWater": false
    },
    "integrator_params": {
      "integrator_type": "LangevinMiddle",
      "Temperature": "300 * kelvin",
      "Friction": "1.0 / picoseconds",
      "Timestep": "0.002 * picoseconds",
      "Pressure": "1.0 * bar"
    },
    "simulation_params": {
      "Ensemble": "NVT",
      "Number of Steps": 5000,
      "record_interval_steps": 100,
      "record_params": ["step", "potentialEnergy", "temperature"]
    }
  }
}
```

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "6BB5",
    "forcefield_files": ["amb

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [4]:
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date and time:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)

registry = agent.path_registry
print('ckpt_dir:',os.path.basename(registry.ckpt_dir))
paths_and_descriptions = registry.list_path_names_and_descriptions()
print("\n".join(paths_and_descriptions.split(",")))

date and time: 2024-09-07
time: 18:57:39
ckpt_dir: ckpt_112
Files found in registry: 1A3N_185459: PDB file downloaded from RSCB
 PDBFile ID: 1A3N_185459
 1A3N_185521: Cleaned File:  Removed Heterogens
 and Water Removed.  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 
 top_sim0_185525: Initial positions for simulation sim0_185525
 sim0_185525: Basic Simulation of Protein 1A3N_185521
 rec0_185526: Simulation trajectory for protein 1A3N_185521 and simulation sim0_185525
 rec1_185526: Simulation state log for protein 1A3N_185521 and simulation sim0_185525
 rec2_185526: Simulation pdb frames for protein 1A3N_185521 and simulation sim0_185525
 6BB5_185644: PDB file downloaded from RSCB
 PDBFile ID: 6BB5_185644
 6BB5_185648: Cleaned File:  Removed Heterogens
 and Water Removed.  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 
 top_sim0_185651: Initial positions

In [5]:
import re
import os

matches = re.findall(r"rec0_\d+", paths_and_descriptions)
file_id1 = matches[0]
file_id2 = matches[1]
traj_path1 = registry.get_mapped_path(file_id1)
traj_path2 = registry.get_mapped_path(file_id2)
assert os.path.exists(traj_path1), "Path does not exist"
assert os.path.exists(traj_path2), "Path does not exist"
print(f"It's asserted that file paths for {file_id1} and {file_id2} exist")

It's asserted that file paths for rec0_185526 and rec0_185652 exist


In [6]:
matches = re.findall(r"top_sim0_\d+", paths_and_descriptions)
file_id1 = matches[0]
file_id2 = matches[1]
top_path1 = registry.get_mapped_path(file_id1)
top_path2 = registry.get_mapped_path(file_id2)
assert os.path.exists(top_path1), "Path does not exist"
assert os.path.exists(top_path2), "Path does not exist"
print(f"It's asserted that file paths for {file_id1} and {file_id2} exist")

It's asserted that file paths for top_sim0_185525 and top_sim0_185651 exist


In [7]:
import mdtraj as md
import numpy as np

traj1 = md.load(traj_path1, top=top_path1)
traj2 = md.load(traj_path2, top=top_path2)

#number of framees, adn total residues simulated
print("Number of frames in traj1: ", traj1.n_frames)
print("Number of frames in traj2: ", traj2.n_frames)
print("Number of residues in traj1: ", traj1.n_residues)
print("Number of residues in traj2: ", traj2.n_residues)


Number of frames in traj1:  60
Number of frames in traj2:  60
Number of residues in traj1:  574
Number of residues in traj2:  284


# Experiment Result: ✅❌

### Completed without Exception or TimeOut Errors ✅
### Attempted all necessary steps ✅
### Completed without Hallucination ✅
### Logic make sense ✅
### Correct Answer ✅